# A simple web crawler

### Loading dependency packages

In [1]:
## scraper packages
!pip install scrapy


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=6802 sha256=afb2afd7d1b06481187fb4af13d1781e8f78de722782a58e5acf71bf952d9e2a
  Stored in directory: c:\users\sajjad\appdata\local\pip\cache\wheels\b2\74\25\517a0ec6186297704db56664268e72686f5cfa8ab398582f33
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11527 sha256=b3dc2547ac446bb0447eb3d5da2c27a795232cc1b4425ac885e57f0a2bd9e7d4
  Stored in directory: c:\users\sajjad\appdata\local\pip\cache\wheels\69\20\28\fbdcea83fadaf56e6e3ed24df2a1b409a8d950d3cce69bbfce
Successfully built protego PyDispatcher
  Attempting uninstall: attrs
    Found existing installati

### Scraping

We use the package `scrapy` for scraping data from web. 

In [2]:
import scrapy


class BrickSetSpider(scrapy.Spider):
    name = 'brick_spider'
    start_urls = ['http://brickset.com/sets/year-2016']

    def parse(self, response):
        SET_SELECTOR = '.set'
        for brickset in response.css(SET_SELECTOR):

            NAME_SELECTOR = 'h1 ::text'
            PIECES_SELECTOR = './/dl[dt/text() = "Pieces"]/dd/a/text()'
            MINIFIGS_SELECTOR = './/dl[dt/text() = "Minifigs"]/dd[2]/a/text()'
            IMAGE_SELECTOR = 'img ::attr(src)'
            yield {
                'name': brickset.css(NAME_SELECTOR).extract_first(),
                'pieces': brickset.xpath(PIECES_SELECTOR).extract_first(),
                'minifigs': brickset.xpath(MINIFIGS_SELECTOR).extract_first(),
                'image': brickset.css(IMAGE_SELECTOR).extract_first(),
            }

        NEXT_PAGE_SELECTOR = '.next a ::attr(href)'
        next_page = response.css(NEXT_PAGE_SELECTOR).extract_first()
        if next_page:
            yield scrapy.Request(
                response.urljoin(next_page),
                callback=self.parse
            )

In [8]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess(settings={
    "FEEDS": {
        "items.json": {"format": "json"},
    },
})

process.crawl(BrickSetSpider)
process.start() # the script will block here until the crawling is finished

2022-02-06 23:02:58 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-02-06 23:02:58 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2o  27 Mar 2018), cryptography 2.2.2, Platform Windows-10-10.0.19041-SP0
2022-02-06 23:02:58 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-02-06 23:02:58 [scrapy.crawler] INFO: Overridden settings:
{}
2022-02-06 23:02:59 [scrapy.extensions.telnet] INFO: Telnet Password: 5e18c325b43c54a3
2022-02-06 23:02:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-02-06 23:02:59 [scrapy.middleware] INFO: Enabled downloader middlewares:
['